In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import pandas_profiling
import warnings
import sklearn
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv(r"C:\Users\ammasi\Desktop\data\churn\churn.csv", sep=",", encoding="utf-8-sig", engine="python")

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
plt.figure(figsize=(10,7)),

(<Figure size 720x504 with 0 Axes>,)

In [5]:
ax=sns.heatmap(df.corr(),vmin=0, vmax=1,annot=True);

In [6]:
df_reg=df.copy().drop(columns=['RowNumber','CustomerId','Surname'])
df_reg.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
# How many unique values are in Columns of Data ?
for i in df_reg.columns:
    if len(df_reg[i].unique())<6:
      print(F'{i}:',len(df_reg[i].unique()),'Values:',df_reg[i].unique())
    else:
      print(F'{i}:',len(df_reg[i].unique()))

CreditScore: 460
Geography: 3 Values: ['France' 'Spain' 'Germany']
Gender: 2 Values: ['Female' 'Male']
Age: 70
Tenure: 11
Balance: 6382
NumOfProducts: 4 Values: [1 3 2 4]
HasCrCard: 2 Values: [1 0]
IsActiveMember: 2 Values: [1 0]
EstimatedSalary: 9999
Exited: 2 Values: [1 0]


In [8]:
df_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [9]:
df_reg.describe().T

,count,mean,std,min,25%,50%,75%,max
CreditScore,10000.0,650.528800,96.653299,350.00,584.00,652.000,718.0000,850.00
Age,10000.0,38.921800,10.487806,18.00,32.00,37.000,44.0000,92.00
Tenure,10000.0,5.012800,2.892174,0.00,3.00,5.000,7.0000,10.00
Balance,10000.0,76485.889288,62397.405202,0.00,0.00,97198.540,127644.2400,250898.09
NumOfProducts,10000.0,1.530200,0.581654,1.00,1.00,1.000,2.0000,4.00
HasCrCard,10000.0,0.705500,0.455840,0.00,0.00,1.000,1.0000,1.00
IsActiveMember,10000.0,0.515100,0.499797,0.00,0.00,1.000,1.0000,1.00
EstimatedSalary,10000.0,100090.239881,57510.492818,11.58,51002.11,100193.915,149388.2475,199992.48
Exited,10000.0,0.203700,0.402769,0.00,0.00,0.000,0.0000,1.00


In [10]:
df_reg.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:
labels = 'Exited', 'Retained'
sizes = [df_reg.Exited[df_reg['Exited']==1].count(), df_reg.Exited[df['Exited']==0].count()]
explode = (0, 0.1)
fig1, ax1 = plt.subplots(figsize=(10, 8))
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')
plt.title("Proportion of customer churned and retained", size = 20)
plt.show()

In [12]:
df_reg['Gender']=pd.get_dummies(df_reg['Gender'],drop_first=True)
Geography_dumies=pd.get_dummies(df_reg['Geography'],drop_first=True)
df_reg=df_reg.drop(columns=['Geography'])
df_reg=pd.concat([df_reg,Geography_dumies],axis=1)
df_reg.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Germany,Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,1


In [13]:
x=df_reg.copy().drop(columns=['Exited']).values
y=df_reg['Exited'].values
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [14]:
print('Train_x',x_train.shape)
print('Train_y',y_train.shape)
print('Test_x',x_test.shape)
print('Test_y',y_test.shape)

Train_x (9000, 11)
Train_y (9000,)
Test_x (1000, 11)
Test_y (1000,)


In [15]:
from sklearn.preprocessing import StandardScaler
obje_ss=StandardScaler()

x_train_ss=obje_ss.fit_transform(x_train)
x_test_ss=obje_ss.fit_transform(x_test)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

model_le=LogisticRegression(random_state=42,max_iter=10000)
model_le.fit(x_train,y_train)
parameters = {'C':[0.8,0.9,1,1.1,1.2]}
model_le_grid = GridSearchCV(model_le, parameters,cv=10,verbose=1,n_jobs=-1).fit(x_train,y_train)
print('Best parameters:',model_le_grid.best_params_)

print('Logistic Regression Train score:',model_le.score(x_train,y_train)*100)
print('Logistic Regression Cros validation score:',model_le_grid.best_score_*100)

Fitting 10 folds for each of 5 candidates, totalling 50 fits
Best parameters: {'C': 0.8}
Logistic Regression Train score: 78.83333333333333
Logistic Regression Cros validation score: 78.81111111111112


In [17]:
model_le=LogisticRegression(random_state=42,C=0.8)
model_le.fit(x_train,y_train)
print('Logistic Regression Train score:',model_le.score(x_train,y_train)*100)

Logistic Regression Train score: 78.83333333333333


In [18]:
from sklearn import svm

model_svc=svm.SVC(random_state=42)
parameters = {'kernel':('linear', 'rbf','poly'), 'C':[0.8,0.9,1,1.1,1.2],'degree':[3,4,5,6]}
model_svc_grid = GridSearchCV(model_svc, parameters,cv=10,verbose=1,n_jobs=-1).fit(x_train_ss,y_train)
print('Best parameters:',model_svc_grid.best_params_)
print('Support Vecktor Classification Cros validation score:',model_svc_grid.best_score_*100)

Fitting 10 folds for each of 60 candidates, totalling 600 fits
Best parameters: {'C': 1, 'degree': 4, 'kernel': 'poly'}
Support Vecktor Classification Cros validation score: 85.63333333333333


In [19]:
model_svc=svm.SVC(random_state=42,C=1,kernel='poly',degree=4)
model_svc.fit(x_train_ss,y_train)
print('Support Vecktor Classification Train score:',model_svc.score(x_train_ss,y_train)*100)

Support Vecktor Classification Train score: 87.27777777777777


In [20]:
from sklearn.ensemble import RandomForestClassifier

model_rfc=RandomForestClassifier(n_jobs=-1)
parameters = {'n_estimators':[50,100,200,300,400],'max_depth':[3,4,5,6]}
model_rfc_grid = GridSearchCV(model_rfc, parameters,cv=10,verbose=1,n_jobs=-1).fit(x_train,y_train)
print(model_rfc_grid.best_params_)

print('Random Forest Classifier Cros validation score:',model_rfc_grid.best_score_*100)

Fitting 10 folds for each of 20 candidates, totalling 200 fits
{'max_depth': 6, 'n_estimators': 300}
Random Forest Classifier Cros validation score: 85.91111111111111


In [21]:
model_rfc=RandomForestClassifier(n_jobs=-1,n_estimators=400,max_depth=6)
model_rfc.fit(x_train,y_train)
print('Random Forest Classifier Train score:',model_rfc.score(x_train,y_train)*100)

Random Forest Classifier Train score: 86.65555555555555


In [22]:
from xgboost import XGBClassifier

model_xgb=XGBClassifier(n_jobs=-1,random_state=42,eval_metric='logloss')
parameters = {'n_estimators':[50,100,200],'max_depth':[3,4,5,6],'learning_rate':[0.1,0.01]}
model_xgb_grid = GridSearchCV(model_xgb, parameters,cv=10,verbose=1).fit(x_train,y_train)
print(model_xgb_grid.best_params_)

print('XGB Classifier Cros validation score:',model_xgb_grid.best_score_*100)

Fitting 10 folds for each of 24 candidates, totalling 240 fits
{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}
XGB Classifier Cros validation score: 86.38888888888889


In [23]:
model_xgb=XGBClassifier(n_jobs=-1,random_state=42,n_estimators=200,max_depth=3,learning_rate=0.1,eval_metric='logloss')
model_xgb.fit(x_train,y_train)
print('XGB Classifier Classifier Train score:',model_xgb.score(x_train,y_train)*100)

XGB Classifier Classifier Train score: 87.9


In [24]:
model_dnn=tf.keras.Sequential()

model_dnn.add(tf.keras.layers.Dense(25,activation='relu',input_dim=11))
model_dnn.add(tf.keras.layers.Dropout(0.5))
model_dnn.add(tf.keras.layers.Dense(10,activation='relu'))
model_dnn.add(tf.keras.layers.Dropout(0.3))
model_dnn.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model_dnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss="binary_crossentropy",metrics=['accuracy'])

In [25]:
model_dnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 25)                300       
_________________________________________________________________
dropout (Dropout)            (None, 25)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                260       
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 571
Trainable params: 571
Non-trainable params: 0
_________________________________________________________________


In [26]:
epoch=200
history=model_dnn.fit(x_train_ss,y_train,epochs=epoch,verbose=0)

In [27]:
print('Loss:',history.history['loss'][-1])
print('Accuracy: %',history.history['accuracy'][-1]*100)

Loss: 0.3812244236469269
Accuracy: % 84.22222137451172


In [28]:
plt.plot(history.history['loss'], label='loss')
plt.xlim([0,epoch])
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.legend()
plt.grid(True)

In [29]:
from sklearn.metrics import classification_report

def test_score(model_name):
  for i in model_name: 
    print(f'{i.__class__} \n{classification_report(y_test,i.predict(x_test))}')

def test_score_ss(model_name):
  for i in model_name:
    print(f'{i.__class__} \n{classification_report(y_test,i.predict(x_test_ss))}') 

liste_test=[model_le,model_rfc,model_xgb]
liste_test_ss=[model_svc]

test_score(liste_test)
test_score_ss(liste_test_ss)

liste_dnn=[]
for i in model_dnn.predict(x_test_ss):
  if i<0.5:
    liste_dnn.append(0)
  else:
    liste_dnn.append(1)
print(f'{model_dnn.__class__} \n{classification_report(y_test,liste_dnn)}')

<class 'sklearn.linear_model._logistic.LogisticRegression'> 
              precision    recall  f1-score   support

           0       0.82      0.98      0.89       809
           1       0.45      0.07      0.13       191

    accuracy                           0.81      1000
   macro avg       0.63      0.53      0.51      1000
weighted avg       0.75      0.81      0.74      1000

<class 'sklearn.ensemble._forest.RandomForestClassifier'> 
              precision    recall  f1-score   support

           0       0.87      0.97      0.92       809
           1       0.77      0.38      0.51       191

    accuracy                           0.86      1000
   macro avg       0.82      0.68      0.71      1000
weighted avg       0.85      0.86      0.84      1000

<class 'xgboost.sklearn.XGBClassifier'> 
              precision    recall  f1-score   support

           0       0.89      0.96      0.92       809
           1       0.74      0.48      0.58       191

    accuracy         